In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 處理找不到元素錯誤
from selenium.common.exceptions import NoSuchElementException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 執行 command 的時候用的
import os

# 執行 Regex
import re

# 隨機取得 User-Agent
from fake_useragent import UserAgent
ua = UserAgent(use_external_data = True)

# requests
import requests as req
from bs4 import BeautifulSoup as bs

# pyautogui
import pyautogui

'''標頭'''
headers = {
    'user-agent': ua.random
}

'''
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
'''
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()               
# my_options.add_argument('--headless')             #不開啟實體瀏覽器背景執行
# my_options.add_argument('--start-maximized')        #最大化視窗
my_options.add_argument('--incognito')              #開啟無痕模式
my_options.add_argument('--disable-popup-blocking') #禁用彈出攔截
my_options.add_argument('--disable-notification')   #取消 chrome 推播通知
my_options.add_argument('--lang=zh-TW')             #設定為正體中文
my_options.add_argument(f'--user-agent={ua.random}') # (Optional)加入 User-Agent


# 使用 Chrome 的 WebDriver
my_service = Service(executable_path='./chromedriver.exe')
driver = webdriver.Chrome(
    options = my_options,
    service = my_service
)

# url
url = "https://transtats.bts.gov/ONTIME/Departures.aspx"

In [ ]:
# 開啟網頁
driver.get(url)

html = driver.page_source
soup = bs(html, "lxml")

# Origin Airport 下拉選單全部欄位
# 共456個
oa_text = [oa.get_text() for oa in soup.select('select#cboAirport > option')]

# Airline 下拉式選單
# 共34個
al_text = [al.get_text() for al in soup.select('select#cboAirline > option')]

# 將下拉式選單組合成list tuple
select_list = [(i, j) for i in oa_text for j in al_text]

# 設定年份
year = 2017

# 計數
sum = 0

# sleep
sleep(1)

# 點選all statistics
driver.find_element(By.CSS_SELECTOR, 'td.btsfont input#chkAllStatistics').click()

# 點選 all Months
month = driver.find_element(By.CSS_SELECTOR, 'td.btsfont input#chkAllMonths')
month.click() 

# 睡一秒
sleep(1)

# 點選 all Days
days = driver.find_element(By.CSS_SELECTOR, 'td.btsfont input#chkAllDays')
days.click()

# 睡一秒
sleep(1)

# 選擇所要years
driver.find_element(By.CSS_SELECTOR, f'table#chkYears input[value="{year}"]').click()

# 設定要跑的筆數
for i in select_list[14365:]:
    try:
        while True:
            sleep(3)

            # Origin Airport下拉式選單
            oa = Select(driver.find_element(By.CSS_SELECTOR, 'td.btsfont > select[name="cboAirport"]'))
            oa.select_by_visible_text(i[0])    
            # Airline 下拉式選單
            al = Select(driver.find_element(By.CSS_SELECTOR, 'td.btsfont > select[name="cboAirline"]'))
            al.select_by_visible_text(i[1])

            # 點選 Submit
            driver.find_element(By.CSS_SELECTOR, 'tr input[type="submit"]').click()
            
            # 計數
            sum += 1
            
            # sleep
            sleep(2)

            print(sum)

            html = driver.page_source
            soup = bs(html, "lxml")


            # 點選 csv download按鈕，判斷如果沒有csv這個元素就跳過。
            if soup.select('td.btsfont > a#DL_CSV') != []:
                driver.find_element(By.CSS_SELECTOR, 'td.btsfont > a#DL_CSV').click()
                sleep(2)

            if sum%50 == 0:
                # 關閉網頁        
                driver.quit()

                # sleep
                sleep(1)

                # 重新開啟網頁
                driver = webdriver.Chrome(
                    options = my_options,
                    service = my_service
                )

                # sleep
                sleep(2)

                # 從新進入
                driver.get(url)

                # sleep
                sleep(5)

                # 點選all statistics
                driver.find_element(By.CSS_SELECTOR, 'td.btsfont input#chkAllStatistics').click()

                # 點選 all Months
                month = driver.find_element(By.CSS_SELECTOR, 'td.btsfont input#chkAllMonths')
                month.click() 
                
                # sleep
                sleep(1)

                # 點選 all Days
                days = driver.find_element(By.CSS_SELECTOR, 'td.btsfont input#chkAllDays')
                days.click()
                
                # sleep
                sleep(1)

                # 選擇所要years
                driver.find_element(By.CSS_SELECTOR, f'table#chkYears input[value="{year}"]').click()
            break
            
    except NoSuchElementException:
        print(f'第{sum}_{i}錯誤')
        driver.get(url)
        
        # sleep
        sleep(3)
                
        html = driver.page_source
        soup = bs(html, "lxml")
        
        # sleep
        sleep(1)
        
        # 點選all statistics
        driver.find_element(By.CSS_SELECTOR, 'td.btsfont input#chkAllStatistics').click()
        
        # 點選 all Months
        month = driver.find_element(By.CSS_SELECTOR, 'td.btsfont input#chkAllMonths')
        month.click() 

        # sleep
        sleep(1)

        # 點選 all Days
        days = driver.find_element(By.CSS_SELECTOR, 'td.btsfont input#chkAllDays')
        days.click()

        # sleep
        sleep(1)

        # 選擇所要years
        driver.find_element(By.CSS_SELECTOR, f'table#chkYears input[value="{year}"]').click()
        
        continue
        
sleep(5)
print(sum)
# 關閉網頁        
driver.quit()